In [20]:
filename = 'vdjdb.slim.txt'
folder = 'vdjdb-2016-09-05/'
outfile = 'trb_sequences'
# This isn't an ideal solution of this problem.
outp = {}
with open(folder+filename, 'r') as inp:
    for line in inp:
        if line.startswith('TRB'):
            data = line.strip().split()
            if data[3] not in outp:
                outp[data[3]]=[data[1]]
            else:
                outp[data[3]].append(data[1])
with open(outfile+'.txt', 'w') as out:
    for i in outp:
        out.write('antigen.epitope:\t'+str(i)+'\t'+str(len(outp[i]))+'\n')
        out.write('cdr3:\t'+'\t'.join(outp[i]))
        out.write('\n//\n')
outp.clear()

In [1]:
import os
def crdir(a):
    if os.path.exists(a)==1:
        pass
    else:
        os.mkdir(a)
    return 0

In [66]:
#From past to fasta
outfile = 'trb_sequences'
folder = 'blast'
crdir(folder)
with open(outfile+'.txt', 'r') as inp:
    allout = open(outfile+'_all', 'w')
    for line in inp:
        if line.startswith('antigen.epitope'):
            data1 = line.strip().split()
            out = open(folder+'/'+outfile+'_'+data1[1]+'_'+data1[2], 'w')
        elif line.startswith('cdr3'):
            seq = set(line.strip().split()[1:])
            name_and_seq = list(map(lambda x, y: '\n'.join(['>'+x, y]), seq, seq))
            all_name_and_seq = list(map(lambda x, y: '\n'.join(['>'+data1[1]+'_'+x, y]), seq, seq))
            out.write('\n'.join(name_and_seq))
            allout.write('\n'.join(all_name_and_seq)+'\n')
        else:
            out.write('\n')
            out.close()
    out.close()
    allout.close()

In [68]:
#check multiple alignment
with open(folder+'/trb_sequences_check', 'r') as trb:
    out = open('trb_sequences_check_align', 'w')
    for line in trb:
        if line.startswith('>'):
            pass
        else:
            out.write(line)
    out.close()

In [129]:
from Bio import AlignIO
from Bio import pairwise2
from Bio import SeqIO
from Bio.SubsMat import MatrixInfo
import time

records = list(SeqIO.parse('trb_sequences_all', 'fasta'))
print(len(records))
iti = 0
#out = open('alignments', 'w')
#out.close()
matrix = MatrixInfo.blosum30
crdir('needle')
#out.write('options:\nmatrix – blosum30\ngap_open = -.1\ngap_extension = -.1\nalignment - needle (global)\n#\n')
for i in range(len(records)):
    out = open('needle/alignments_'+str(i), 'w')
    out.write('options:\nmatrix – blosum30\ngap_open = -.1\ngap_extension = -.1\nalignment - needle (global)\n#\n')
    out.write('seq:\t'+records[i].id+'\n')
    #print(str(i)+'/'+str(len(records)))
    for k in range(len(records)-i-1):
        out.write('seq_2:\t'+records[k+1].id+'\nalignments:\n')
        seq1 = records[i].seq
        seq2 = records[k+1].seq
        alignment = pairwise2.align.globalds(seq1, seq2, matrix, -.1, -.1)
        #alignments - [(seq1, seq2, weight, start, end),...]
        for m in alignment:
            subs = -2*m[4]+len(seq1)+len(seq2)
            for x in range(len(m[0])):
                if m[0][x] != m[1][x]:
                    #print(m[0][x], m[1][x], subs)
                    subs += 1
            out.write(m[0]+'\n'+m[1]+'\nscore:\t'+str(m[2])+'\n')
            out.write('ins\tdel\tsub\n')
            out.write(str(m[4]-len(seq1))+'\t'+str(m[4]-len(seq2))+'\t'+str(subs)+'\n//\n')
    out.close()
    time.sleep(2)
#print(pairwise2.__doc__)

3528


In [151]:
import glob
crdir('group_pair_alignment')

def multiple_pairwise_alignment(gapstart, gapexten, matrix, path):
    files = glob.glob(path+'/trb_sequences*')
    for i in files:
        if int(i.split('_')[3]) >= 5:
            records = list(SeqIO.parse(i, 'fasta'))
            outfolder = 'group_pair_alignment/'+matrix+'_'+str(gapstart)+'_'+str(gapexten)
            crdir(outfolder)
            out = open(outfolder+'/'+i.split('/')[1], 'w')
            out.write('options:\nmatrix – ' + matrix+ '\ngap_open = ' + str(gapstart) +
                      '\ngap_extension = ' + str(gapexten) + '\nalignment - needle (global)\n#\n')
            for i in range(len(records)):
                out.write('seq:\t'+records[i].id+'\n')
                for k in range(len(records)-i-1):
                    out.write('seq_2:\t'+records[k+1].id+'\nalignments:\n')
                    seq1 = records[i].seq
                    seq2 = records[k+1].seq
                    alignment = pairwise2.align.globalds(seq1, seq2, matrices[matrix], gapstart, gapexten)
                    for m in alignment:
                        subs = -2*m[4]+len(seq1)+len(seq2)
                        for x in range(len(m[0])):
                            if m[0][x] != m[1][x]:
                                subs += 1
                        out.write(m[0]+'\n'+m[1]+'\nscore:\t'+str(m[2])+'\n')
                        out.write('ins\tdel\tsub\tidentity\n')
                        ins = m[4]-len(seq1)
                        dell = m[4]-len(seq2)
                        ident = (float(m[4]-ins-dell-subs))/m[4]
                        out.write(str(m[4]-len(seq1))+'\t'+str(m[4]-len(seq2))+'\t'+str(subs)+'\t'+str(ident)+'\n//\n')
            out.close()

In [152]:
def mindict(dic):
    i = None
    for obj in dic:
        if i == None:
            i = dic[obj]
        else:
            if i >= dic[obj]:
                i = dic[obj]
            for obj in dic:
                if i >= dic[obj]:
                    i = dic[obj]
    return i

In [158]:
folder = 'blast'
#, 'blosum95':MatrixInfo.blosum95
matrices = {'blosum30':MatrixInfo.blosum30, 'blosum62':MatrixInfo.blosum62}
for matrix in matrices:
    gap_start = [float("{0:.3f}".format(float(mindict(matrices[matrix]))*2/3)), float("{0:.3f}".format(float(mindict(matrices[matrix]))/2))]
    for gaps in gap_start:
        gap_exten = [gaps, float("{0:.3f}".format(float(gaps)/2)), -.1]
        for gape in gap_exten:
            multiple_pairwise_alignment(gaps, gape, matrix, folder)
            if gape == -.1 and matrix != 'blosum30' and gaps == float(mindict(matrices[matrix]))/2:
                multiple_pairwise_alignment(-.1, gape, matrix, folder)

In [3]:
matrices = {'blosum30':MatrixInfo.blosum30}
multiple_pairwise_alignment(-.1, -.1, 'blosum30', 'blast')

NameError: name 'MatrixInfo' is not defined

In [2]:
with open('trb_sequences_all_teach', 'r') as inp:
    sequences_with_antigen = {}
    for line in inp:
        if line.startswith('>'):
            antigen = line.strip().split('_')[0].replace('>','')
        else:
            if line.strip() in sequences_with_antigen:
                sequences_with_antigen[line.strip()].append(antigen)
            else:
                sequences_with_antigen[line.strip()] = [antigen]
    print(len(sequences_with_antigen))
with open('trb_sequences_for_groovy', 'w') as out:
    for i in sequences_with_antigen:
        out.write(i+'\n')

3467


In [32]:
import os
for subs in range(4):
    for dels in range(4):
        for ins in range(4):
            total = [subs+dels+ins]
            for tot in total:
                #os.system("groovy TreeSearch.groovy trb_sequences_all_teach "+str(subs)+","+str(dels)+","+str(ins)+","+str(tot)+" sub_del_ins_tot/out_teach_"+str(subs)+"_"+str(dels)+"_"+str(ins)+"_"+str(tot))
                pass

In [8]:
import os
dels, ins, subs, tot = 7, 0, 0, 7
os.system("groovy TreeSearch.groovy trb_sequences_all_teach "+str(dels)+","+str(ins)+","+str(subs)+","+str(tot)+" sub_del_ins_tot/out_"+str(dels)+"_"+str(ins)+"_"+str(subs)+"_"+str(tot))

0

In [22]:
import json
import urllib.request as urllib2

data = []
req = open('aaa_test.txt', 'r')
data = json.loads(req.read())

print(data.keys())

dict_keys(['nodes', 'links'])


In [ ]:
N=len(data['nodes'])
N

In [30]:
import igraph as ig
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

ImportError: No module named 'igraph'